In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Directorio de datos
data_dir = '../../database/happy_train'

# Parámetros
img_width, img_height = 150, 150
batch_size = 32
epochs = 10

# Preprocesamiento de datos
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Cargar y dividir datos en entrenamiento y validación
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='grayscale',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='grayscale',
    subset='validation')

# Definición del modelo CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilación del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Guardar el modelo entrenado
model.save('facial_recognition_model.h5')


In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

data_dir = '../../database/db_emociones'
categorias = ['happy', 'angry']
img_size = 100
data = []
labels = []
image_count = {categoria: 0 for categoria in categorias}

for categoria in categorias:
    path = os.path.join(data_dir, categoria)
    label = categorias.index(categoria)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array, (img_size, img_size))
        data.append(img_array)
        labels.append(label)
        image_count[categoria] += 1

data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0

print("Cantidad de imágenes por clase:")
for categoria, count in image_count.items():
    print(f"{categoria}: {count}")

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_split=0.1)

test_loss, test_acc = model.evaluate(X_test, y_test)
predictions = model.predict(X_test)

model.save('modelo_emociones.h5')


Cantidad de imágenes por clase:
happy: 1825
angry: 960
Epoch 1/10
63/63 [==============================] - 38s 563ms/step - loss: 0.6485 - accuracy: 0.6454 - val_loss: 0.6485 - val_accuracy: 0.6726
Epoch 2/10
63/63 [==============================] - 32s 506ms/step - loss: 0.6137 - accuracy: 0.6628 - val_loss: 0.7488 - val_accuracy: 0.6726
Epoch 3/10
63/63 [==============================] - 35s 550ms/step - loss: 0.5871 - accuracy: 0.7002 - val_loss: 0.5788 - val_accuracy: 0.7130
Epoch 4/10
63/63 [==============================] - 35s 553ms/step - loss: 0.5231 - accuracy: 0.7421 - val_loss: 0.5570 - val_accuracy: 0.7130
Epoch 5/10
63/63 [==============================] - 35s 549ms/step - loss: 0.4732 - accuracy: 0.7611 - val_loss: 0.5979 - val_accuracy: 0.7085
Epoch 6/10
63/63 [==============================] - 34s 534ms/step - loss: 0.4275 - accuracy: 0.7970 - val_loss: 0.5807 - val_accuracy: 0.7220
Epoch 7/10
63/63 [==============================] - 31s 490ms/step - loss: 0.3687 - acc

C:\Users\Familia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Cargar el modelo entrenado
model = load_model('./modelo_emociones.h5')  # Reemplaza con la ruta a tu modelo

# Inicializar la cámara
cap = cv2.VideoCapture(0)  # Puedes cambiar el número si tienes varias cámaras

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocesar el fotograma
    img = cv2.resize(frame, (100, 100))  # Ajusta el tamaño a la entrada del modelo
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # Realizar la predicción
    predictions = model.predict(img)
    label = "Alegre" if predictions[0][0] > predictions[0][1] else "Triste"
    color = (0, 255, 0) if label == "Alegre" else (0, 0, 255)

    # Mostrar el resultado en la ventana de la cámara
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
